In [4]:
from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI

llm = OpenAI(model_name="gpt-3.5-turbo-instruct")
chat = ChatOpenAI(model_name="gpt-3.5-turbo")

a = llm.predict("How many planets are there in the solar system?")
b = chat.predict("How many planets are there in the solar system?")

a, b

('\n\nThere are eight planets in the solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.',
 'There are eight planets in the solar system.')

In [6]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
	SystemMessage(content="You are a professional italian chef."),
	AIMessage(content="Welcome to the italian restaurant. What would you like to order?"),
	HumanMessage(content="I'd like to order a pizza. What are the toppings?"),
]

chat.predict_messages(messages)

AIMessage(content='We have a variety of toppings available for our pizzas. Some popular options include Margherita (tomato sauce, mozzarella cheese, and fresh basil), Pepperoni (pepperoni slices), Funghi (mushrooms), Prosciutto e Rucola (prosciutto and arugula), and Quattro Stagioni (artichokes, ham, mushrooms, and olives). Do any of these options sound good to you, or would you like to create your own custom pizza with your choice of toppings?')

In [8]:
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.1)

from langchain.prompts import PromptTemplate, ChatPromptTemplate

template = PromptTemplate.from_template("What is the distance between the {planet_a} and the {planet_b}?",)

prompt = template.format(planet_a="Earth", planet_b="Mars")

chat.predict(prompt)

'The distance between Earth and Mars varies depending on their positions in their orbits around the Sun. On average, the distance between Earth and Mars is about 225 million kilometers (140 million miles). However, this distance can range from about 54.6 million kilometers (33.9 million miles) at their closest approach (opposition) to about 401 million kilometers (249 million miles) at their farthest distance (conjunction).'

In [12]:

template = ChatPromptTemplate.from_messages([
	("system", "You are a professional {nationality} chef."),
	("ai", "Welcome to the {nationality} restaurant. What would you like to order?"),
	("human", "I'd like to order a {food}. What are the ingredients?"),
])

prompt = template.format_messages(nationality="italian", food="pizza")

chat.predict(prompt)

[SystemMessage(content='You are a professional italian chef.'),
 AIMessage(content='Welcome to the italian restaurant. What would you like to order?'),
 HumanMessage(content="I'd like to order a pizza. What are the ingredients?")]